* Load PMI matrix
* Load dictionary
* Calculate scores
* Create plots

In [1]:
import numpy as np
def load_pmi(path):
    pmi = np.load(path)
    return pmi

import pickle
def load_dictionary(path):
    fname = open(path, 'rb')
    data = pickle.load(fname)
    return data

def calc_score_v2(pmi, d, sentence_a, sentence_b):
    s = 0
    pairs = 0
    for word_1 in sentence_a.split():
        for word_2 in sentence_b.split():
            x = d.token2id.get(word_1,-1)
            y = d.token2id.get(word_2,-1)
            if x == -1 or y == -1:
                continue
            s += pmi[x][y]
            pairs += 1
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.true_divide(s, pairs)
    return c

In [4]:
def get_all_scores(pmi, d, all_sentences, k):
    successive = []
    for i in range(len(all_sentences) - k):
        successive.append(calc_score_v2(pmi, d, all_sentences[i], all_sentences[i+k]))
    return successive

In [5]:
def get_all_sentences(data):
    return [line for line in data.split('\n') if len(line)>0]

In [6]:
def read_txt(fname):
    with open(fname, 'r') as f:
        all = f.read()
    return all

In [7]:
import os
import pandas as pd

In [81]:
from numpy import dot
from numpy.linalg import norm
def cos_sim(a, b):
#     with np.errstate(divide='ignore', invalid='ignore'):
#         c = np.true_divide(dot(a, b), norm(a)*norm(b))
#     return c
    return dot(a, b)/(norm(a)*norm(b))

In [57]:
def iterator(path):
    for fx in os.listdir(path):
        if fx.endswith('.pkl'):
            d = load_dictionary(path+fx)
            
        if fx.endswith('_non_norm.npy'):
            pmi = load_pmi(path+fx)
            
        if fx.endswith('lexical.txt'):
            data = read_txt(path+fx)
            sentences = get_all_sentences(data)
            
    return pmi, d, sentences

In [9]:
def generator(path, k):
    pmi, d, sentences = iterator(path)
    parent_dir = os.path.basename(os.path.dirname(path))
    name = parent_dir + '_lexical.npy'
    
    lexical = get_all_scores(pmi, d, sentences, k)
    np.save(path + name, lexical)
    
    name = name = parent_dir + '_lexical'
    df = pd.DataFrame(lexical)
    df.to_csv(path + name+'.csv')
    print("Created {}".format(name))

In [10]:
def process_v2(fname):
    all_data = get_data(fname)
    all_data = unidecode.unidecode(all_data)
    sentences = make_sentences(all_data)
    clean_sentences = []
    removed_sentences = []
    for i, sentence in enumerate(sentences):
        t = remove_punc_clean(sentence)
        if len(t) > 0:
            clean_sentences.append(t)
        else:
            removed_sentences.append(i)

    # write_to_file_lexical(clean_sentences, fname)
    print('Done processing', fname)
    return removed_sentences

In [43]:
from itertools import combinations
def generate_ssm(pmi, d, sentences):
    slen = len(sentences)
    ssm = np.ones((slen, slen))
    for i, j in combinations(range(slen), 2):
        score = calc_score_v2(pmi, d, sentences[i], sentences[j])
        ssm[i, j] = score
        ssm[j, i] = score
    return ssm

In [55]:
def similarity_bwn_word_vectors(pmi, d, sentence_a, sentence_b):
    s = 0
    pairs = 0
    for word_1 in sentence_a.split():
        for word_2 in sentence_b.split():
            x = d.token2id.get(word_1,-1)
            y = d.token2id.get(word_2,-1)
            if x == -1 or y == -1:
                continue
            s += cos_sim(pmi[x], pmi[y])
            pairs += 1
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.true_divide(s, pairs)
    return c

In [96]:
def generate_ssm_lex_vectors(pmi, d, sentences):
    slen = len(sentences)
    ssm = np.ones((slen, slen))
    for i, j in combinations(range(slen), 2):
        score = similarity_bwn_word_vectors(pmi, d, sentences[i], sentences[j])
#         print('{} {} {:.2f}'.format(i,j, score))
        ssm[i, j] = score
        ssm[j, i] = score
    return ssm

In [11]:
def lexical_mean_vectors(pmi, d, sentences):
    mean_vectors = []
    for sentence in sentences:
        sentence_embed = []
        for word in sentence.split():
            word_index = d.token2id.get(word, -1)
            sentence_embed.append(pmi[word_index])

        mat_ = np.asarray(sentence_embed)
    
        mean_vectors.append(np.mean(mat_, axis = 0 ))
    return np.asarray(mean_vectors)
    
        

In [12]:
def lexical_max_vectors(pmi, d, sentences):
    max_vectors = []
    for sentence in sentences:
        sentence_embed = []
        for word in sentence.split():
            word_index = d.token2id.get(word, -1)
            sentence_embed.append(pmi[word_index])

        mat_ = np.asarray(sentence_embed)
    
        max_vectors.append(np.max(mat_, axis = 0 ))
    return np.asarray(max_vectors)
    

In [18]:
def save_mat(path, out_path = path):
    for fx in os.listdir(path):
        if fx.endswith('.npy'):
            name = fx[:-4]
            print("Loaded "+fx)
            embed = np.load(path+fx)
            # print(embed)

            # sns.histplot(embed)
            book, method = get_embed_method_and_name(name)

            title = book.title() + ' ' + label(method)
            savemat(out_path+title+'.mat', mdict={'values': embed})

In [41]:
path = '../final/lexical results/a christmas carol/'
# k = 1
# generator(path, k)

pmi, d, sentences = iterator(path)
mean_vectors = lexical_mean_vectors(pmi, d, sentences)
np.save(path+'lexical_mean_vectors.npy', mean_vectors)

max_vectors = lexical_max_vectors(pmi, d, sentences)
np.save(path+'lexical_max_vectors.npy', mean_vectors)

Created a christmas carol_lexical


In [26]:
path = '../final/lexical results/a tale of 2 cities/'
k = 1
generator(path, k)

Created a tale of 2 cities_lexical.npy


In [27]:
path = '../final/lexical results/great gatsby/'
k = 1
generator(path, k)

Created great gatsby_lexical.npy


In [50]:
path = '../final/lexical results/metamorphosis/'
# k = 1
# generator(path, k)


pmi, d, sentences = iterator(path)
ssm = generate_ssm(pmi, d, sentences)
np.save(path+'new_ssmv2.npy', ssm)


# mean_vectors = lexical_mean_vectors(pmi, d, sentences)
# np.save(path+'lexical_mean_vectors.npy', mean_vectors)

# max_vectors = lexical_max_vectors(pmi, d, sentences)
# np.save(path+'lexical_max_vectors.npy', mean_vectors)

In [29]:
path = '../final/lexical results/mysterious affair/'
k = 1
generator(path, k)

Created mysterious affair_lexical.npy


In [30]:
path = '../final/lexical results/pride and prejudice/'
k = 1
generator(path, k)

Created pride and prejudice_lexical.npy


In [52]:
path = '../final/lexical results/the prophet/'
# k = 1
# generator(path, k)

pmi, d, sentences = iterator(path)
ssm = generate_ssm(pmi, d, sentences)
np.save(path+'new_ssm_v2.npy', ssm)


# mean_vectors = lexical_mean_vectors(pmi, d, sentences)
# np.save(path+'lexical_mean_vectors.npy', mean_vectors)

# max_vectors = lexical_max_vectors(pmi, d, sentences)
# np.save(path+'lexical_max_vectors.npy', mean_vectors)

In [75]:
path = '../final/lexical results/heart of darkness/'
# k = 1
# generator(path, k)

pmi, d, sentences = iterator(path)
ssm = generate_ssm(pmi, d, sentences)
np.save(path+'new_ssm_v2.npy', ssm)

# mean_vectors = lexical_mean_vectors(pmi, d, sentences)
# np.save(path+'lexical_mean_vectors.npy', mean_vectors)
# savemat(out_path+'.mat', mdict={'values': embed})

# max_vectors = lexical_max_vectors(pmi, d, sentences)
# np.save(path+'lexical_max_vectors.npy', mean_vectors)

KeyboardInterrupt: 

In [ ]:
path = '../final/lexical results/a christmas carol/'
for fx in os.listdir(path):
    if fx.endswith('.txt'):
        data = read_txt(path+fx)
        indices = process_v2()
        sentences = get_all_sentences(data)
            

In [54]:
path = '../final/lexical results/a christmas carol/'
pmi_carol, d_carol, sentences_carol = iterator(path)
# mean_vectors = lexical_mean_vectors(pmi_carol, d_carol, sentences_carol)

# np.save(path+'lexical_mean_vectors.npy', mean_vectors)
ssm = generate_ssm(pmi_carol, d_carol, sentences_carol)
np.save(path+'new_ssm_v2.npy', ssm)


In [15]:
max_vectors = lexical_max_vectors(pmi_carol, d_carol, sentences_carol)
np.save(path+'lexical_max_vectors.npy', mean_vectors)

In [63]:
path = '../final/lexical results/a christmas carol/'

pmi, d, sentences = iterator(path)
ssm = generate_ssm_lex_vectors(pmi, d, sentences)
np.save(path+'new_ssm_lex_vect.npy', ssm)

In [64]:
path = '../final/lexical results/metamorphosis/'

pmi, d, sentences = iterator(path)
ssm = generate_ssm_lex_vectors(pmi, d, sentences)
np.save(path+'new_ssm_lex_vect.npy', ssm)

In [78]:
path = '../final/lexical results/heart of darkness/'

pmi, d, sentences = iterator(path)
ssm = generate_ssm_lex_vectors(pmi, d, sentences)
np.save(path+'new_ssm_lex_vect2.npy', ssm)

0 1 0.87
0 2 0.87
0 3 0.91
0 4 0.90
0 5 0.91
0 6 0.94
0 7 0.90
0 8 0.85
0 9 0.93
0 10 0.91
0 11 0.89
0 12 0.90
0 13 0.89
0 14 0.92
0 15 0.92
0 16 0.91
0 17 0.91
0 18 0.93
0 19 0.90
0 20 0.93
0 21 0.91
0 22 0.90
0 23 0.89
0 24 0.91
0 25 0.89
0 26 0.89
0 27 0.88
0 28 0.88
0 29 0.89
0 30 0.89
0 31 0.91
0 32 0.89
0 33 0.87
0 34 0.91
0 35 0.90
0 36 0.89
0 37 0.91
0 38 0.88
0 39 0.90
0 40 0.86
0 41 0.91
0 42 0.85
0 43 0.88
0 44 0.87
0 45 0.87
0 46 0.90
0 47 0.83
0 48 0.90
0 49 0.91
0 50 0.91
0 51 0.89
0 52 0.88
0 53 0.80
0 54 0.90
0 55 0.86
0 56 0.87
0 57 0.87
0 58 0.92
0 59 0.89
0 60 0.87
0 61 0.90
0 62 0.89
0 63 0.90
0 64 0.84
0 65 0.91
0 66 0.89
0 67 0.84
0 68 0.91
0 69 0.90
0 70 0.90
0 71 0.90
0 72 0.92
0 73 0.85
0 74 0.92
0 75 0.93
0 76 0.88
0 77 0.92
0 78 0.91
0 79 0.93
0 80 0.91
0 81 0.87
0 82 0.90
0 83 0.90
0 84 0.93
0 85 0.92
0 86 0.90
0 87 0.87
0 88 0.88
0 89 0.88
0 90 0.92
0 91 0.87
0 92 0.90
0 93 0.89
0 94 0.85
0 95 0.90
0 96 0.89
0 97 0.89
0 98 0.85
0 99 0.93
0 100 0.89
0 101 0.

0 768 0.88
0 769 0.92
0 770 0.87
0 771 0.84
0 772 0.89
0 773 0.91
0 774 0.93
0 775 0.88
0 776 0.91
0 777 0.90
0 778 0.89
0 779 0.85
0 780 0.90
0 781 0.91
0 782 0.77
0 783 0.91
0 784 0.85
0 785 0.91
0 786 0.93
0 787 0.91
0 788 0.89
0 789 0.92
0 790 0.91
0 791 0.92
0 792 0.87
0 793 0.92
0 794 0.89
0 795 0.90
0 796 0.93
0 797 0.87
0 798 0.91
0 799 0.87
0 800 0.90
0 801 0.93
0 802 0.91
0 803 0.90
0 804 0.88
0 805 0.86
0 806 0.87
0 807 0.90
0 808 0.88
0 809 0.89
0 810 0.90
0 811 0.88
0 812 0.91
0 813 0.92
0 814 0.89
0 815 0.91
0 816 0.83
0 817 0.79
0 818 0.88
0 819 0.84
0 820 0.90
0 821 0.87
0 822 0.91
0 823 0.90
0 824 0.92
0 825 0.90
0 826 0.89
0 827 0.89
0 828 0.94
0 829 0.92
0 830 0.90
0 831 0.87
0 832 0.89
0 833 0.92
0 834 0.90
0 835 0.93
0 836 0.88
0 837 0.89
0 838 0.95
0 839 0.86
0 840 0.92
0 841 0.86
0 842 0.84
0 843 0.91
0 844 0.93
0 845 0.89
0 846 0.89
0 847 0.91
0 848 0.89
0 849 0.89
0 850 0.92
0 851 0.89
0 852 0.91
0 853 0.89
0 854 0.90
0 855 0.87
0 856 0.90
0 857 0.89
0 858 0.90

0 1477 0.88
0 1478 0.90
0 1479 0.91
0 1480 0.92
0 1481 0.91
0 1482 0.91
0 1483 0.94
0 1484 0.87
0 1485 0.89
0 1486 0.91
0 1487 0.86
0 1488 0.90
0 1489 0.95
0 1490 0.88
0 1491 0.86
0 1492 0.91
0 1493 0.91
0 1494 0.89
0 1495 0.87
0 1496 0.89
0 1497 0.91
0 1498 0.91
0 1499 0.89
0 1500 0.91
0 1501 0.88
0 1502 0.89
0 1503 0.90
0 1504 0.86
0 1505 0.91
0 1506 0.90
0 1507 0.89
0 1508 0.94
0 1509 0.89
0 1510 0.87
0 1511 0.88
0 1512 0.92
0 1513 0.89
0 1514 0.91
0 1515 0.88
0 1516 0.91
0 1517 0.83
0 1518 0.87
0 1519 0.94
0 1520 0.89
0 1521 0.86
0 1522 0.85
0 1523 0.80
0 1524 0.90
0 1525 0.89
0 1526 0.87
0 1527 0.90
0 1528 0.89
0 1529 0.95
0 1530 0.92
0 1531 0.89
0 1532 0.92
0 1533 0.92
0 1534 0.89
0 1535 0.91
0 1536 0.94
0 1537 0.92
0 1538 0.92
0 1539 0.84
0 1540 0.91
0 1541 0.90
0 1542 0.84
0 1543 0.85
0 1544 0.77
0 1545 0.90
0 1546 0.91
0 1547 0.88
0 1548 0.87
0 1549 0.88
0 1550 0.92
0 1551 0.90
0 1552 0.90
0 1553 0.84
0 1554 0.91
0 1555 0.92
0 1556 0.89
0 1557 0.92
0 1558 0.88
0 1559 0.86
0 15

0 2189 0.91
0 2190 0.92
0 2191 0.92
0 2192 0.91
0 2193 0.84
0 2194 0.94
0 2195 0.88
0 2196 0.86
0 2197 0.93
0 2198 0.94
0 2199 0.91
0 2200 0.93
0 2201 0.94
0 2202 0.88
0 2203 0.94
0 2204 0.90
0 2205 0.90
0 2206 0.90
0 2207 0.92
0 2208 0.92
0 2209 0.87
0 2210 0.88
0 2211 0.89
0 2212 0.94
0 2213 0.91
0 2214 0.88
0 2215 0.90
0 2216 0.95
0 2217 0.90
0 2218 0.89
0 2219 0.88
0 2220 0.88
0 2221 0.75
0 2222 0.89
0 2223 0.91
0 2224 0.89
0 2225 0.89
0 2226 0.89
0 2227 0.91
0 2228 0.86
0 2229 0.93
0 2230 0.93
0 2231 0.90
0 2232 0.94
0 2233 0.91
0 2234 0.91
0 2235 0.95
0 2236 0.90
0 2237 0.91
0 2238 0.90
0 2239 0.88
0 2240 0.93
0 2241 0.91
0 2242 0.89
0 2243 0.92
0 2244 0.93
0 2245 0.91
0 2246 0.89
0 2247 0.88
0 2248 0.88
0 2249 0.94
0 2250 0.88
0 2251 0.83
0 2252 0.92
0 2253 0.93
0 2254 0.89
0 2255 0.89
0 2256 0.92
0 2257 0.88
0 2258 0.85
0 2259 0.94
0 2260 0.95
0 2261 0.91
0 2262 0.90
0 2263 0.91
0 2264 0.91
0 2265 0.90
0 2266 0.90
0 2267 0.93
0 2268 0.93
0 2269 0.89
0 2270 0.92
0 2271 0.85
0 22

1 567 0.88
1 568 0.90
1 569 0.87
1 570 0.84
1 571 0.88
1 572 0.90
1 573 0.87
1 574 0.90
1 575 0.87
1 576 0.82
1 577 0.88
1 578 0.87
1 579 0.90
1 580 0.89
1 581 0.89
1 582 0.91
1 583 0.88
1 584 0.90
1 585 0.90
1 586 0.92
1 587 0.89
1 588 0.89
1 589 0.89
1 590 0.90
1 591 0.90
1 592 0.88
1 593 0.88
1 594 0.91
1 595 0.88
1 596 0.90
1 597 0.88
1 598 0.88
1 599 0.88
1 600 0.87
1 601 0.90
1 602 0.89
1 603 0.89
1 604 0.89
1 605 0.89
1 606 0.88
1 607 0.88
1 608 0.89
1 609 0.85
1 610 0.87
1 611 0.91
1 612 0.87
1 613 0.90
1 614 0.90
1 615 0.86
1 616 0.92
1 617 0.86
1 618 0.88
1 619 0.92
1 620 0.90
1 621 0.88
1 622 0.87
1 623 0.87
1 624 0.90
1 625 0.91
1 626 0.89
1 627 0.83
1 628 0.86
1 629 0.88
1 630 0.89
1 631 0.87
1 632 0.87
1 633 0.88
1 634 0.89
1 635 0.89
1 636 0.80
1 637 0.88
1 638 0.85
1 639 0.89
1 640 0.91
1 641 0.88
1 642 0.88
1 643 0.91
1 644 0.90
1 645 0.90
1 646 0.89
1 647 0.88
1 648 0.87
1 649 0.89
1 650 0.87
1 651 0.87
1 652 0.90
1 653 0.87
1 654 0.89
1 655 0.88
1 656 0.89
1 657 0.87

...

4 1468 0.90
4 1469 0.90
4 1470 0.93
4 1471 0.91
4 1472 0.90
4 1473 0.90
4 1474 0.88
4 1475 0.88
4 1476 0.91
4 1477 0.88
4 1478 0.90
4 1479 0.91
4 1480 0.92
4 1481 0.91
4 1482 0.90
4 1483 0.92
4 1484 0.87
4 1485 0.89
4 1486 0.91
4 1487 0.88
4 1488 0.90
4 1489 0.94
4 1490 0.89
4 1491 0.88
4 1492 0.90
4 1493 0.91
4 1494 0.89
4 1495 0.88
4 1496 0.89
4 1497 0.90
4 1498 0.91
4 1499 0.89
4 1500 0.91
4 1501 0.88
4 1502 0.90
4 1503 0.91
4 1504 0.87
4 1505 0.91
4 1506 0.90
4 1507 0.88
4 1508 0.93
4 1509 0.89
4 1510 0.87
4 1511 0.88
4 1512 0.91
4 1513 0.90
4 1514 0.90
4 1515 0.89
4 1516 0.90
4 1517 0.84
4 1518 0.88
4 1519 0.92
4 1520 0.89
4 1521 0.86
4 1522 0.86
4 1523 0.82
4 1524 0.89
4 1525 0.89
4 1526 0.87
4 1527 0.90
4 1528 0.89
4 1529 0.93
4 1530 0.91
4 1531 0.89
4 1532 0.92
4 1533 0.92
4 1534 0.89
4 1535 0.91
4 1536 0.91
4 1537 0.92
4 1538 0.91
4 1539 0.86
4 1540 0.91
4 1541 0.90
4 1542 0.85
4 1543 0.86
4 1544 0.79
4 1545 0.90
4 1546 0.91
4 1547 0.88
4 1548 0.88
4 1549 0.89
4 1550 0.92
4 15

4 2173 0.90
4 2174 0.91
4 2175 0.90
4 2176 0.90
4 2177 0.91
4 2178 0.92
4 2179 0.91
4 2180 0.91
4 2181 0.92
4 2182 0.91
4 2183 0.93
4 2184 0.89
4 2185 0.90
4 2186 0.90
4 2187 0.91
4 2188 0.88
4 2189 0.89
4 2190 0.91
4 2191 0.90
4 2192 0.90
4 2193 0.86
4 2194 0.92
4 2195 0.88
4 2196 0.87
4 2197 0.92
4 2198 0.93
4 2199 0.91
4 2200 0.92
4 2201 0.91
4 2202 0.89
4 2203 0.91
4 2204 0.89
4 2205 0.89
4 2206 0.90
4 2207 0.92
4 2208 0.91
4 2209 0.87
4 2210 0.88
4 2211 0.89
4 2212 0.93
4 2213 0.91
4 2214 0.88
4 2215 0.90
4 2216 0.93
4 2217 0.89
4 2218 0.89
4 2219 0.89
4 2220 0.89
4 2221 0.78
4 2222 0.88
4 2223 0.91
4 2224 0.89
4 2225 0.89
4 2226 0.89
4 2227 0.90
4 2228 0.87
4 2229 0.92
4 2230 0.92
4 2231 0.90
4 2232 0.91
4 2233 0.91
4 2234 0.91
4 2235 0.94
4 2236 0.90
4 2237 0.91
4 2238 0.89
4 2239 0.89
4 2240 0.93
4 2241 0.91
4 2242 0.89
4 2243 0.91
4 2244 0.91
4 2245 0.90
4 2246 0.89
4 2247 0.89
4 2248 0.88
4 2249 0.93
4 2250 0.88
4 2251 0.85
4 2252 0.91
4 2253 0.91
4 2254 0.89
4 2255 0.88
4 22

5 565 0.91
5 566 0.89
5 567 0.90
5 568 0.93
5 569 0.90
5 570 0.86
5 571 0.92
5 572 0.92
5 573 0.90
5 574 0.92
5 575 0.89
5 576 0.83
5 577 0.90
5 578 0.88
5 579 0.92
5 580 0.92
5 581 0.92
5 582 0.93
5 583 0.91
5 584 0.92
5 585 0.92
5 586 0.95
5 587 0.91
5 588 0.91
5 589 0.92
5 590 0.93
5 591 0.92
5 592 0.91
5 593 0.89
5 594 0.93
5 595 0.90
5 596 0.92
5 597 0.90
5 598 0.90
5 599 0.91
5 600 0.90
5 601 0.92
5 602 0.91
5 603 0.91
5 604 0.91
5 605 0.92
5 606 0.92
5 607 0.91
5 608 0.91
5 609 0.87
5 610 0.89
5 611 0.94
5 612 0.90
5 613 0.92
5 614 0.92
5 615 0.89
5 616 0.95
5 617 0.88
5 618 0.91
5 619 0.94
5 620 0.92
5 621 0.90
5 622 0.90
5 623 0.90
5 624 0.92
5 625 0.94
5 626 0.92
5 627 0.85
5 628 0.89
5 629 0.91
5 630 0.93
5 631 0.89
5 632 0.90
5 633 0.90
5 634 0.91
5 635 0.92
5 636 0.81
5 637 0.91
5 638 0.87
5 639 0.91
5 640 0.94
5 641 0.90
5 642 0.90
5 643 0.93
5 644 0.93
5 645 0.93
5 646 0.92
5 647 0.90
5 648 0.90
5 649 0.91
5 650 0.89
5 651 0.89
5 652 0.93
5 653 0.89
5 654 0.91
5 655 0.90

5 1315 0.90
5 1316 0.90
5 1317 0.90
5 1318 0.92
5 1319 0.90
5 1320 0.94
5 1321 0.90
5 1322 0.92
5 1323 0.89
5 1324 0.90
5 1325 0.91
5 1326 0.91
5 1327 0.92
5 1328 0.91
5 1329 0.92
5 1330 0.89
5 1331 0.89
5 1332 0.89
5 1333 0.92
5 1334 0.91
5 1335 0.93
5 1336 0.91
5 1337 0.92
5 1338 0.91
5 1339 0.89
5 1340 0.91
5 1341 0.90
5 1342 0.91
5 1343 0.92
5 1344 0.94
5 1345 0.90
5 1346 0.91
5 1347 0.90
5 1348 0.93
5 1349 0.91
5 1350 0.92
5 1351 0.92
5 1352 0.89
5 1353 0.89
5 1354 0.92
5 1355 0.92
5 1356 0.92
5 1357 0.91
5 1358 0.93
5 1359 0.91
5 1360 0.90
5 1361 0.92
5 1362 0.90
5 1363 0.87
5 1364 0.93
5 1365 0.93
5 1366 0.91
5 1367 0.89
5 1368 0.91
5 1369 0.92
5 1370 0.93
5 1371 0.93
5 1372 0.92
5 1373 0.90
5 1374 0.92
5 1375 0.90
5 1376 0.91
5 1377 0.91
5 1378 0.90
5 1379 0.93
5 1380 0.89
5 1381 0.89
5 1382 0.90
5 1383 0.90
5 1384 0.93
5 1385 0.93
5 1386 0.94
5 1387 0.91
5 1388 0.92
5 1389 0.95
5 1390 0.90
5 1391 0.92
5 1392 0.91
5 1393 0.91
5 1394 0.89
5 1395 0.92
5 1396 0.86
5 1397 0.91
5 13

KeyboardInterrupt: 

In [82]:
cos_sim(pmi[485], pmi[0])

<ipython-input-81-f608ea089872>:7: RuntimeWarning: invalid value encountered in double_scalars
  return dot(a, b)/(norm(a)*norm(b))


nan

In [85]:
cos_sim(pmi[0], pmi[1])

<ipython-input-81-f608ea089872>:7: RuntimeWarning: invalid value encountered in double_scalars
  return dot(a, b)/(norm(a)*norm(b))


nan

In [89]:
for i in range(len(pmi)):
    print('Sentence {}, norm {:.2f}'.format(i+1, norm(pmi[i])))

Sentence 1, norm 0.00
Sentence 2, norm 124.24
Sentence 3, norm 95.14
Sentence 4, norm 94.22
Sentence 5, norm 172.60
Sentence 6, norm 145.76
Sentence 7, norm 96.16
Sentence 8, norm 93.41
Sentence 9, norm 124.67
Sentence 10, norm 94.04
Sentence 11, norm 93.88
Sentence 12, norm 94.09
Sentence 13, norm 172.04
Sentence 14, norm 94.74
Sentence 15, norm 93.51
Sentence 16, norm 93.10
Sentence 17, norm 93.65
Sentence 18, norm 172.23
Sentence 19, norm 92.95
Sentence 20, norm 93.56
Sentence 21, norm 217.12
Sentence 22, norm 125.36
Sentence 23, norm 144.52
Sentence 24, norm 159.78
Sentence 25, norm 95.03
Sentence 26, norm 123.97
Sentence 27, norm 94.37
Sentence 28, norm 94.35
Sentence 29, norm 124.88
Sentence 30, norm 182.14
Sentence 31, norm 144.36
Sentence 32, norm 95.98
Sentence 33, norm 94.54
Sentence 34, norm 93.94
Sentence 35, norm 93.63
Sentence 36, norm 94.98
Sentence 37, norm 124.86
Sentence 38, norm 93.52
Sentence 39, norm 181.75
Sentence 40, norm 159.85
Sentence 41, norm 160.14
Sentence

Sentence 803, norm 124.22
Sentence 804, norm 124.09
Sentence 805, norm 160.17
Sentence 806, norm 96.60
Sentence 807, norm 124.78
Sentence 808, norm 124.16
Sentence 809, norm 93.74
Sentence 810, norm 93.54
Sentence 811, norm 94.23
Sentence 812, norm 181.94
Sentence 813, norm 125.09
Sentence 814, norm 213.96
Sentence 815, norm 94.42
Sentence 816, norm 125.80
Sentence 817, norm 93.76
Sentence 818, norm 266.15
Sentence 819, norm 94.28
Sentence 820, norm 124.04
Sentence 821, norm 125.09
Sentence 822, norm 93.04
Sentence 823, norm 92.98
Sentence 824, norm 159.73
Sentence 825, norm 204.95
Sentence 826, norm 123.96
Sentence 827, norm 93.02
Sentence 828, norm 190.56
Sentence 829, norm 92.78
Sentence 830, norm 93.08
Sentence 831, norm 171.45
Sentence 832, norm 92.85
Sentence 833, norm 145.31
Sentence 834, norm 93.76
Sentence 835, norm 160.75
Sentence 836, norm 204.75
Sentence 837, norm 160.31
Sentence 838, norm 94.31
Sentence 839, norm 94.75
Sentence 840, norm 172.38
Sentence 841, norm 94.11
Sen

Sentence 1626, norm 92.95
Sentence 1627, norm 159.52
Sentence 1628, norm 92.97
Sentence 1629, norm 124.20
Sentence 1630, norm 92.85
Sentence 1631, norm 198.00
Sentence 1632, norm 92.85
Sentence 1633, norm 331.71
Sentence 1634, norm 93.60
Sentence 1635, norm 190.63
Sentence 1636, norm 145.81
Sentence 1637, norm 94.10
Sentence 1638, norm 296.00
Sentence 1639, norm 144.54
Sentence 1640, norm 124.93
Sentence 1641, norm 124.85
Sentence 1642, norm 159.99
Sentence 1643, norm 93.32
Sentence 1644, norm 92.74
Sentence 1645, norm 93.82
Sentence 1646, norm 93.59
Sentence 1647, norm 93.57
Sentence 1648, norm 124.92
Sentence 1649, norm 242.38
Sentence 1650, norm 92.92
Sentence 1651, norm 92.85
Sentence 1652, norm 93.09
Sentence 1653, norm 93.37
Sentence 1654, norm 172.02
Sentence 1655, norm 124.04
Sentence 1656, norm 92.95
Sentence 1657, norm 93.06
Sentence 1658, norm 124.19
Sentence 1659, norm 309.56
Sentence 1660, norm 93.12
Sentence 1661, norm 159.77
Sentence 1662, norm 92.85
Sentence 1663, norm 

Sentence 2378, norm 93.20
Sentence 2379, norm 94.03
Sentence 2380, norm 94.04
Sentence 2381, norm 125.01
Sentence 2382, norm 93.40
Sentence 2383, norm 92.95
Sentence 2384, norm 93.39
Sentence 2385, norm 271.36
Sentence 2386, norm 93.16
Sentence 2387, norm 124.32
Sentence 2388, norm 93.81
Sentence 2389, norm 92.85
Sentence 2390, norm 145.04
Sentence 2391, norm 93.20
Sentence 2392, norm 93.21
Sentence 2393, norm 160.78
Sentence 2394, norm 252.18
Sentence 2395, norm 92.85
Sentence 2396, norm 144.62
Sentence 2397, norm 160.21
Sentence 2398, norm 93.87
Sentence 2399, norm 93.60
Sentence 2400, norm 159.36
Sentence 2401, norm 93.60
Sentence 2402, norm 144.42
Sentence 2403, norm 93.44
Sentence 2404, norm 93.12
Sentence 2405, norm 171.60
Sentence 2406, norm 93.91
Sentence 2407, norm 93.73
Sentence 2408, norm 181.93
Sentence 2409, norm 159.93
Sentence 2410, norm 124.63
Sentence 2411, norm 144.46
Sentence 2412, norm 159.67
Sentence 2413, norm 124.13
Sentence 2414, norm 93.58
Sentence 2415, norm 9

Sentence 3030, norm 93.17
Sentence 3031, norm 92.89
Sentence 3032, norm 124.60
Sentence 3033, norm 124.54
Sentence 3034, norm 144.98
Sentence 3035, norm 92.85
Sentence 3036, norm 92.78
Sentence 3037, norm 92.92
Sentence 3038, norm 252.76
Sentence 3039, norm 144.38
Sentence 3040, norm 190.46
Sentence 3041, norm 198.21
Sentence 3042, norm 93.90
Sentence 3043, norm 248.56
Sentence 3044, norm 93.27
Sentence 3045, norm 125.23
Sentence 3046, norm 94.36
Sentence 3047, norm 93.19
Sentence 3048, norm 93.36
Sentence 3049, norm 93.05
Sentence 3050, norm 124.04
Sentence 3051, norm 190.78
Sentence 3052, norm 93.15
Sentence 3053, norm 144.62
Sentence 3054, norm 190.55
Sentence 3055, norm 145.17
Sentence 3056, norm 144.91
Sentence 3057, norm 93.20
Sentence 3058, norm 92.99
Sentence 3059, norm 93.63
Sentence 3060, norm 159.48
Sentence 3061, norm 93.27
Sentence 3062, norm 191.11
Sentence 3063, norm 124.40
Sentence 3064, norm 93.06
Sentence 3065, norm 93.29
Sentence 3066, norm 92.85
Sentence 3067, norm 

Sentence 3765, norm 92.93
Sentence 3766, norm 93.63
Sentence 3767, norm 124.70
Sentence 3768, norm 93.19
Sentence 3769, norm 124.27
Sentence 3770, norm 93.74
Sentence 3771, norm 124.57
Sentence 3772, norm 264.68
Sentence 3773, norm 144.36
Sentence 3774, norm 159.74
Sentence 3775, norm 94.04
Sentence 3776, norm 94.19
Sentence 3777, norm 124.09
Sentence 3778, norm 159.59
Sentence 3779, norm 172.39
Sentence 3780, norm 94.88
Sentence 3781, norm 125.17
Sentence 3782, norm 144.63
Sentence 3783, norm 123.97
Sentence 3784, norm 94.53
Sentence 3785, norm 93.96
Sentence 3786, norm 124.13
Sentence 3787, norm 182.37
Sentence 3788, norm 93.91
Sentence 3789, norm 124.99
Sentence 3790, norm 172.13
Sentence 3791, norm 125.24
Sentence 3792, norm 236.94
Sentence 3793, norm 328.41
Sentence 3794, norm 198.11
Sentence 3795, norm 93.06
Sentence 3796, norm 145.43
Sentence 3797, norm 93.70
Sentence 3798, norm 93.12
Sentence 3799, norm 93.49
Sentence 3800, norm 123.89
Sentence 3801, norm 125.27
Sentence 3802, 

In [95]:
pmi[]

'60'

In [69]:
np.seterr(over='raise')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [66]:
path = '../final/lexical results/the prophet/'

pmi, d, sentences = iterator(path)
ssm = generate_ssm_lex_vectors(pmi, d, sentences)
np.save(path+'new_ssm_lex_vect.npy', ssm)

In [77]:
import gc
gc.collect()

4567

In [67]:
cos_sim(0,0)

<ipython-input-62-48ba7b7474a0>:4: RuntimeWarning: invalid value encountered in true_divide
  return dot(a, b)/(norm(a)*norm(b))


nan

In [68]:
1+1

2